# * VINSIGHT : Daily Status

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Aggregate : Timestamp
->  AGG_PERF_NEWCO

In [13]:
# Input parameter
v_start_date = 20250201
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ PPN_TM, AREA_TYPE
        , COUNT(DISTINCT METRIC_CD) CNT_METRIC, COUNT(1) ROW_CNT
        , MIN(TM_KEY_DAY) MIN_DT, MAX(TM_KEY_DAY) MAX_DT, MAX(ACTUAL_AS_OF) LAST_ACTUAL
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE CENTER IN ('Revenue', 'Sales', 'Profitability')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY PPN_TM, AREA_TYPE
    ORDER BY PPN_TM, AREA_TYPE
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    vis_timestamp_df = chk_src_df.copy()
    print(f'\n{vis_timestamp_df.to_string(max_cols=7)}') #max_rows=1000
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> v_start_date: 20250201

Current Timestamp...

                PPN_TM AREA_TYPE  CNT_METRIC  ROW_CNT    MIN_DT    MAX_DT  LAST_ACTUAL
0  2025-03-05 08:39:02         C         530    16960  20250201  20250304     20250303
1  2025-03-05 08:39:02         G         317    81152  20250201  20250304     20250303
2  2025-03-05 08:39:02         P         341    10912  20250201  20250304     20250303
3  2025-03-05 08:39:02         Z          29     6496  20250201  20250304     20250302
4  2025-03-05 08:41:07         C           4      128  20250201  20250304     20250304
5  2025-03-05 08:41:07         G           4     1024  20250201  20250304     20250304
6  2025-03-05 08:41:07         P           4      128  20250201  20250304     20250304
7  2025-03-05 10:46:31         H         316   667392  20250201  20250304     20250303
8  2025-03-05 10:50:15         H           4     8448  20250201  20250304     20250304
9  2025-03-05 12:46:34        HH         258   792576  20

## Data Source : Ex-T Revenue
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [27]:
# Input parameter
v_start_date = 20250201
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , CAST(SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) AS DECIMAL(18)) C
        , CAST(SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) AS DECIMAL(18)) P
        , CAST(SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) AS DECIMAL(18)) G
        , CAST(SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) AS DECIMAL(18)) H
        , CAST(SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) AS DECIMAL(18)) HH
        , CAST(SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) AS DECIMAL(18)) CCAA
        , CAST(SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) AS DECIMAL(18)) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    WHERE REMARK IN ('salisa', 'jutar11')
    AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME
    ORDER BY REMARK, METRIC_CD
""")


try:
    # Get : Ex-T Revenue
    src_cur.execute(query)
    rows = src_cur.fetchall()
    print(f'\nCurrent Ex-T Revenue...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    src_t_revenue_df = chk_src_df.copy()
    mod_col_list = src_t_revenue_df.iloc[:, 7:12].columns.tolist()
    for col in mod_col_list:
        src_t_revenue_df[col] = src_t_revenue_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{src_t_revenue_df.to_string(max_cols=16)}')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> v_start_date: 20250201

Current Ex-T Revenue...

     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE  START_DAY   END_DAY     C              P              G              H             HH           CCAA CCAATT  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-05 04:53:19   20250201  20250302  None  2,476,273,934  2,470,321,307  2,470,307,446  2,470,307,446  2,459,929,765   None              6         1105    33150
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-05 04:49:11   20250201  20250302  None    280,382,865    275,418,722    275,418,722    275,418,722            nan   None              5          177     5310
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-05 04:51:34   20250201  20250302  None  2,195,891,069  2,194,902,585  2,194,888,724  2,194,888,724            nan   None              5          177     5310
3   jutar11     TB1S000500           Pre